In [18]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [19]:
import os

# Set device
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
if "mps" == device: os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = "1"

In [20]:
import gc

gc.collect()

def clear_cache():
    gc.collect()
    if (torch.cuda.is_available()):
        torch.cuda.empty_cache()
    if (torch.backends.mps.is_available()):
        torch.mps.empty_cache()

clear_cache()    

In [21]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [22]:
model_name='google/flan-t5-base'

# Removed torch_dtype=torch.bfloat16 as this one does not work on MPS
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [23]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [24]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

original_model = original_model.to(device)

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"].to(device), 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

-------------------------------------------------------------------

In [25]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [26]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [27]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
})


In [28]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'


# 25 steps takes about 18m on Macbook Pro M1 Max for fine-tuning
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=5,
    use_mps_device=True
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

/Users/surya/Documents/personal/kaggle/venv_kaggle/lib/python3.11/site-packages/transformers/training_args.py:1877: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(


In [29]:
trainer.train()

  0%|          | 0/25 [00:00<?, ?it/s]

{'loss': 50.1082, 'learning_rate': 9.600000000000001e-06, 'epoch': 0.06}
{'loss': 48.6112, 'learning_rate': 9.200000000000002e-06, 'epoch': 0.12}
{'loss': 46.1425, 'learning_rate': 8.8e-06, 'epoch': 0.19}
{'loss': 44.7414, 'learning_rate': 8.400000000000001e-06, 'epoch': 0.25}
{'loss': 43.4294, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.31}
{'loss': 43.0684, 'learning_rate': 7.600000000000001e-06, 'epoch': 0.38}
{'loss': 41.5881, 'learning_rate': 7.2000000000000005e-06, 'epoch': 0.44}
{'loss': 40.1531, 'learning_rate': 6.800000000000001e-06, 'epoch': 0.5}
{'loss': 40.0276, 'learning_rate': 6.4000000000000006e-06, 'epoch': 0.56}
{'loss': 39.0772, 'learning_rate': 6e-06, 'epoch': 0.62}
{'loss': 39.2866, 'learning_rate': 5.600000000000001e-06, 'epoch': 0.69}
{'loss': 36.9582, 'learning_rate': 5.2e-06, 'epoch': 0.75}
{'loss': 37.0335, 'learning_rate': 4.800000000000001e-06, 'epoch': 0.81}
{'loss': 35.6999, 'learning_rate': 4.4e-06, 'epoch': 0.88}
{'loss': 37.0885, 'learning_rate': 

TrainOutput(global_step=25, training_loss=38.72052917480469, metrics={'train_runtime': 1126.094, 'train_samples_per_second': 0.178, 'train_steps_per_second': 0.022, 'train_loss': 38.72052917480469, 'epoch': 1.56})

In [30]:
trainer.save_model(output_dir=output_dir)

In [31]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained(output_dir).to(device)

In [32]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs_ids = tokenizer(prompt, return_tensors="pt").input_ids
print(dash_line)
print('ORIGINAL MODEL RUN')
original_model_outputs = original_model.generate(input_ids=input_ids.to(device), generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print('FINE TUNED MODEL RUN')
instruct_model_outputs = instruct_model.generate(input_ids=input_ids.to(device), generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')

---------------------------------------------------------------------------------------------------
ORIGINAL MODEL RUN
---------------------------------------------------------------------------------------------------
FINE TUNED MODEL RUN
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
#Person1: Have you considered upgrading your software? #Person2: I'm not sure what exactly I would need. #Person1: I'd like to make sure I have enough power to make up my own flyers and banners. #Person2: How can I make up my own flyers and banners? #Person2: I'd like to make up my own flyers and banners. #Person2: I'm not sure what exactly I would need. #Person1: I'm not sure what I would need. #Person2: I'm not sure. I'm not s

## Evaluate the model qunatitatively (Using ROUGE Metric)

In [33]:
rouge = evaluate.load('rouge')

In [34]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []

for i, dialogue in enumerate(dialogues):
    print(i)
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    original_model_outputs = original_model.generate(input_ids=input_ids.to(device), generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids.to(device), generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)
    
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df

0
1
2
3
4
5
6
7
8
9


,human_baseline_summaries,original_model_summaries,instruct_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,This memo should be distributed to all employe...,This memo is to be distributed to all employee...
1,In order to prevent employees from wasting tim...,The memo is intended to be distributed to all ...,This memo is to be distributed to all employee...
2,Ms. Dawson takes a dictation for #Person1# abo...,Messages to employees should be sent to the of...,This memo is to be distributed to all employee...
3,#Person2# arrives late because of traffic jam....,Taking public transport to work would be a goo...,Taking public transport to work is a good idea.
4,#Person2# decides to follow #Person1#'s sugges...,I'm a traffic jam.,Taking public transport to work is a good idea.
5,#Person2# complains to #Person1# about the tra...,#Person1: You're finally here!,Taking public transport to work is a good idea.
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
7,#Person1# tells Kate that Masha and Hero are g...,#Person1: #Person2: #Person1: #Person2: #Perso...,Masha and Hero are getting divorced.
8,#Person1# and Kate talk about the divorce betw...,"#Person1: Well, I still can't believe it.",Masha and Hero are getting divorced.
9,#Person1# and Brian are at the birthday party ...,Brian's birthday is coming to an end.,Brian's birthday is coming up.


In [35]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.2070316008855029, 'rouge2': 0.06244201648456967, 'rougeL': 0.18421592489051006, 'rougeLsum': 0.18693129552152107}
INSTRUCT MODEL:
{'rouge1': 0.28647748040489973, 'rouge2': 0.13497482028216662, 'rougeL': 0.23619027925479535, 'rougeLsum': 0.23930701616185485}


## Parameter Efficient Fine-Tuning (PEFT)

### Setup the PEFT/LoRA model for Fine-Tuning

In [36]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [37]:
peft_model = get_peft_model(original_model, 
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


### Train PEFT Adapter

In [38]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=5,
    logging_steps=1,
    max_steps=1    
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

In [39]:
peft_trainer.train()

peft_model_path="./peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

  0%|          | 0/1 [00:00<?, ?it/s]

{'loss': 34.202, 'learning_rate': 0.0, 'epoch': 0.06}
{'train_runtime': 164.5097, 'train_samples_per_second': 0.049, 'train_steps_per_second': 0.006, 'train_loss': 34.20197296142578, 'epoch': 0.06}


('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/spiece.model',
 './peft-dialogue-summary-checkpoint-local/added_tokens.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

In [40]:
from peft import PeftModel, PeftConfig

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")


peft_model = PeftModel.from_pretrained(peft_model_base, 
                                       './peft-dialogue-summary-checkpoint-local/', 
                                       is_trainable=False)

print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0
all model parameters: 251116800
percentage of trainable model parameters: 0.00%


### Evaluation of Model Quality (fine-tuned vs base)

In [41]:
index = 200
dialogue = dataset['test'][index]['dialogue']
baseline_human_summary = dataset['test'][index]['summary']

peft_model = peft_model.to(device)

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
print(dash_line)
print('ORIGINAL MODEL')
original_model_outputs = original_model.generate(input_ids=input_ids.to(device), generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print('INSTRUCT MODEL')
instruct_model_outputs = instruct_model.generate(input_ids=input_ids.to(device), generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print('PEFT MODEL')
peft_model_outputs = peft_model.generate(input_ids=input_ids.to(device), generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
ORIGINAL MODEL
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL
---------------------------------------------------------------------------------------------------
PEFT MODEL
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
#Person1#: Have you considered upgrading your system? #Person2: Yes, but I'm not sure what exactly I would like to upgrade. #Person1: I'm not sure what exactly I would like to upgrade. #Person1: You could also consider adding a painting program to your software.
--------------------------------------------------------------------

### Evaluate the Model Quantitatively (with ROUGE Metric)

In [42]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    print(idx)
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    human_baseline_text_output = human_baseline_summaries[idx]
    
    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])
df

0
1
2
3
4
5
6
7
8
9


,human_baseline_summaries,original_model_summaries,instruct_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,This memo is to be distributed to all employee...,This memo is to be distributed to all employee...,The memo is to be distributed to all employees...
1,In order to prevent employees from wasting tim...,The following memos are in the works for emplo...,This memo is to be distributed to all employee...,The memo is to be distributed to all employees...
2,Ms. Dawson takes a dictation for #Person1# abo...,Employees are being warned that they will be f...,This memo is to be distributed to all employee...,The memo is to be distributed to all employees...
3,#Person2# arrives late because of traffic jam....,People are complaining about the traffic jams ...,Taking public transport to work is a good idea.,The traffic jam at the Carrefour intersection ...
4,#Person2# decides to follow #Person1#'s sugges...,The commuters are talking about the commute.,Taking public transport to work is a good idea.,The traffic jam at the Carrefour intersection ...
5,#Person2# complains to #Person1# about the tra...,I'm here again.,Taking public transport to work is a good idea.,The traffic jam at the Carrefour intersection ...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are divorced.,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
8,#Person1# and Kate talk about the divorce betw...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.,Masha and Hero are getting divorced.
9,#Person1# and Brian are at the birthday party ...,"Happy birthday, Brian.",Brian's birthday is coming up.,Brian's birthday is coming up.


As you can see above, the PEFT Model is giving almost same results as the instruct_model with a much smaller amount of parameters fine-tuned.